In [2]:
# ==============================================================================
# 1_data_engineering_weather.ipynb
# ==============================================================================
# Título: Pipeline de Enriquecimento de Dados Meteorológicos (ETL)
# Autor: Time Data Science
# Descrição:
#   Este notebook executa o processo de ETL (Extração, Transformação e Carga) para
#   enriquecer o conjunto de dados de voos com dados climáticos históricos da
#   API Open-Meteo. A estratégia utiliza a hora prevista de partida para evitar
#   o vazamento de dados (Data Leakage) e garante a integridade através do
#   tratamento de valores nulos e fuso horário.
# ==============================================================================

import pandas as pd
import numpy as np
import requests
import time
import os

# Configuracoes Globais
DATASET_VOOS = 'BrFlights2.csv'
DATASET_CLIMA_RAW = 'weather_history_2015_2017.csv'
DATASET_FINAL = 'BrFlights_Enriched_v4.csv'
API_URL = "https://archive-api.open-meteo.com/v1/archive"

# ==============================================================================
# 1. CARGA E PREPARACAO INICIAL DOS DADOS DE VOOS
# ==============================================================================
print("Iniciando Pipeline de Engenharia de Dados...")

if not os.path.exists(DATASET_VOOS):
    print("Erro Critico: Arquivo 'BrFlights2.csv' nao encontrado. Faca o upload.")
else:
    print("Carregando dataset de voos: BrFlights2.csv...")
    df = pd.read_csv(DATASET_VOOS, encoding='latin1', low_memory=False)

    # Selecao de colunas criticas para o georreferenciamento e cruzamento
    cols_geo = ['Aeroporto.Origem', 'LatOrig', 'LongOrig', 'Partida.Prevista']
    df_geo = df[cols_geo].dropna().copy()

    # Conversao para datetime e arredondamento para hora cheia (Chave de Juncao)
    df_geo['Partida.Prevista'] = pd.to_datetime(df_geo['Partida.Prevista'])
    df_geo['data_hora_clima'] = df_geo['Partida.Prevista'].dt.floor('h')

    print(f"Dataset carregado. Total de registros: {len(df)}")

# ==============================================================================
# 2. ANALISE DE PARETO E OTIMIZACAO DE CHAMADAS
# ==============================================================================
print("\nOtimizando estrategia de extracao (Pareto 90%)...")

# Identificar aeroportos que concentram 90% do trafego para otimizar chamadas de API
top_aeroportos = df['Aeroporto.Origem'].value_counts().reset_index()
top_aeroportos.columns = ['Aeroporto', 'Voos']
top_aeroportos['Acumulado'] = (top_aeroportos['Voos'] / top_aeroportos['Voos'].sum()).cumsum()

# Selecao dos aeroportos principais
lista_vips = top_aeroportos[top_aeroportos['Acumulado'] <= 0.90]['Aeroporto'].tolist()

print(f"Total de Aeroportos: {len(top_aeroportos)}")
print(f"Aeroportos selecionados (VIPs): {len(lista_vips)} (Cobrem 90% do trafego)")

# Obter coordenadas unicas apenas para os aeroportos selecionados
coords_vip = df[df['Aeroporto.Origem'].isin(lista_vips)] \
    .groupby('Aeroporto.Origem')[['LatOrig', 'LongOrig']] \
    .first() \
    .reset_index()

# ==============================================================================
# 3. EXTRACAO DE DADOS (EXTRACT) - OPEN-METEO API
# ==============================================================================
# Verifica se ja existe um cache local para evitar custos de processamento e API
if os.path.exists(DATASET_CLIMA_RAW):
    print(f"\nArquivo de clima '{DATASET_CLIMA_RAW}' ja existe. Pulando download.")
    df_weather_final = pd.read_csv(DATASET_CLIMA_RAW)
else:
    print(f"\nIniciando download historico (2015-2017) para {len(coords_vip)} aeroportos...")
    weather_dfs = []
    start_time = time.time()

    for index, row in coords_vip.iterrows():
        airport = row['Aeroporto.Origem']

        # Parametros da API (Precipitacao e Vento sao os principais para atrasos)
        params = {
            "latitude": row['LatOrig'],
            "longitude": row['LongOrig'],
            "start_date": "2015-01-01",
            "end_date": "2017-12-31",
            "hourly": "precipitation,wind_speed_10m",
            "timezone": "UTC"
        }

        try:
            r = requests.get(API_URL, params=params)
            if r.status_code == 200:
                data = r.json()
                temp_df = pd.DataFrame({
                    'time': data['hourly']['time'],
                    'precipitation': data['hourly']['precipitation'],
                    'wind_speed': data['hourly']['wind_speed_10m']
                })
                temp_df['Aeroporto.Origem'] = airport
                weather_dfs.append(temp_df)
                print(f"Baixado: {airport} ({index+1}/{len(coords_vip)})")
            elif r.status_code == 429:
                print(f"Limite de taxa atingido em {airport}. Aguardando 10s...")
                time.sleep(10)
            else:
                print(f"Erro {r.status_code} em {airport}")

        except Exception as e:
            print(f"Excecao em {airport}: {str(e)}")

        # Pausa para respeitar os limites da API gratuita
        time.sleep(0.5)

    if weather_dfs:
        df_weather_final = pd.concat(weather_dfs, ignore_index=True)
        df_weather_final.to_csv(DATASET_CLIMA_RAW, index=False)
        print(f"Download concluido. Tempo total: {time.time() - start_time:.1f}s")
    else:
        print("Falha critica: Nenhum dado meteorologico foi baixado.")

# ==============================================================================
# 4. FUSAO E TRATAMENTO (TRANSFORM)
# ==============================================================================
print("\nExecutando Juncao (Voo + Clima) baseada em Hora PREVISTA...")

# Garantir tipos de dados compativeis para o cruzamento
df['Partida.Prevista'] = pd.to_datetime(df['Partida.Prevista'])
df['data_hora_merge'] = df['Partida.Prevista'].dt.floor('h')

df_weather_final['time'] = pd.to_datetime(df_weather_final['time'], utc=True)

# Juncao a esquerda (Left Join)
df_enriched = df.merge(
    df_weather_final,
    left_on=['Aeroporto.Origem', 'data_hora_merge'],
    right_on=['Aeroporto.Origem', 'time'],
    how='left'
)

# Identificar onde o clima foi inventado (Imputado) ---
# Criamos uma flag: 1 se o dado nao existia na API, 0 se era real
df_enriched['clima_imputado'] = df_enriched['precipitation'].isnull().astype(int)

# Imputacao Inteligente por Estado ---
print("Imputando nulos por mediana regional (Estado)...")

# Primeiro tentamos preencher com a mediana de chuva e vento de cada Estado
df_enriched['precipitation'] = df_enriched['precipitation'].fillna(
    df_enriched.groupby('Estado.Origem')['precipitation'].transform('median')
)
df_enriched['wind_speed'] = df_enriched['wind_speed'].fillna(
    df_enriched.groupby('Estado.Origem')['wind_speed'].transform('median')
)

# Se ainda sobrarem nulos (estados sem cobertura), usamos a mediana nacional ou zero
fill_values = {
    'precipitation': 0.0,
    'wind_speed': df_enriched['wind_speed'].median()
}
df_enriched.fillna(value=fill_values, inplace=True)

# Limpeza de colunas redundantes
if 'time' in df_enriched.columns:
    df_enriched.drop(columns=['time'], inplace=True)

print("✅ Transformacao concluida com imputacao regional e flag de controle.")

# ==============================================================================
# 5. CARGA FINAL (LOAD)
# ==============================================================================
print(f"\nSalvando Dataset Enriquecido: {DATASET_FINAL}...")

# Exportacao do arquivo final para uso no modelo preditivo
df_enriched.to_csv(DATASET_FINAL, index=False)

print("-" * 50)
print("PIPELINE DE ENGENHARIA CONCLUIDO COM SUCESSO")
print(f"Dataset Original: {df.shape}")
print(f"Dataset Enriquecido: {df_enriched.shape}")
print("Novas Features Integradas: precipitation, wind_speed")
print("-" * 50)

# Exibicao de amostra para validacao
display(df_enriched[['Aeroporto.Origem', 'Partida.Prevista', 'precipitation', 'wind_speed']].head())

Iniciando Pipeline de Engenharia de Dados...
Carregando dataset de voos: BrFlights2.csv...
Dataset carregado. Total de registros: 2542519

Otimizando estrategia de extracao (Pareto 90%)...
Total de Aeroportos: 189
Aeroportos selecionados (VIPs): 44 (Cobrem 90% do trafego)

Iniciando download historico (2015-2017) para 44 aeroportos...
Baixado: Aeroporto Internacional Do Rio De Janeiro/Galeao (1/44)
Baixado: Afonso Pena (2/44)
Baixado: Bahia - Jorge Amado (3/44)
Baixado: Brigadeiro Lysias Rodrigues (4/44)
Baixado: Buenos Aires (5/44)
Baixado: Buenos Aires/Aeroparque (6/44)
Baixado: Campo Grande (7/44)
Baixado: Cataratas (8/44)
Baixado: Congonhas (9/44)
Baixado: Deputado Luis Eduardo Magalhaes (10/44)
Baixado: Eduardo Gomes (11/44)
Baixado: Eurico De Aguiar Salles (12/44)
Baixado: Governador Aluizio Alves (13/44)
Baixado: Governador Jorge Teixeira De Oliveira (14/44)
Baixado: Governador Jose Richa (15/44)
Baixado: Guararapes - Gilberto Freyre (16/44)
Baixado: Guarulhos - Governador Andre

,Aeroporto.Origem,Partida.Prevista,precipitation,wind_speed
0,Afonso Pena,2016-01-30 08:58:00+00:00,0.0,9.1
1,Salgado Filho,2016-01-13 12:13:00+00:00,0.0,1.0
2,Salgado Filho,2016-01-29 12:13:00+00:00,0.0,6.0
3,Salgado Filho,2016-01-19 12:13:00+00:00,0.0,4.1
4,Salgado Filho,2016-01-30 12:13:00+00:00,0.0,5.9
